# 07_market_analytics.ipynb

В этом ноутбуке реализуются основные функции для HR-аналитики, которые будут использоваться в Telegram-боте:

- Подсчёт средних/медианных зарплат по регионам, опыту, навыкам
- Топовые компании по рынку и по зарплатам
- Топ востребованных навыков в профессии/городе/грейде
- Сравнение вакансии с рынком

Все функции возвращают таблицы или словари, которые можно легко интегрировать в бота.

In [1]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [3]:
import pandas as pd
import duckdb
import plotly.express as px
import os

In [80]:
# Подключение к базе DuckDB уже сделано выше
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

In [81]:
# 1. Чтение основной таблицы вакансий
df_vacancy = con.execute("SELECT * FROM vacancy").fetchdf()
print(f"Вакансий: {len(df_vacancy)}")
print("Пример вакансии:")
print(df_vacancy.head(2))

# 2. Таблица связей вакансий и навыков (многие-ко-многим)
df_vacancy_skill = con.execute("SELECT * FROM vacancy_skill").fetchdf()
print(f"\nСвязей vacancy-skill: {len(df_vacancy_skill)}")
print(df_vacancy_skill.head(2))

# 3. Справочник навыков (skills)
df_skill = con.execute("SELECT * FROM skill").fetchdf()
print(f"\nУникальных навыков: {len(df_skill)}")
print(df_skill.head(2))

# 4. Чтение справочника городов/регионов (если нужен)
df_area = pd.read_csv('/content/drive/MyDrive/hh-hr-bot/data/areas_hh.csv')
print(f"\nГородов/регионов: {len(df_area)}")
print(df_area.head(2))


Вакансий: 3067
Пример вакансии:
          id                               title              published_at  \
0  120682290       Водитель с личным автомобилем  2025-05-19T09:30:49+0300   
1  120761341  Middle/Senior Frontend разработчик  2025-05-20T13:03:11+0300   

                                         description  salary_from  salary_to  \
0  <p>Вакансия &quot;Водитель с личным автомобиле...          NaN    90000.0   
1  <p><strong>О компании и команде</strong></p> <...          NaN     3000.0   

  salary_currency  experience_hh  area_id  \
0             RUR    Более 6 лет        3   
1             USD  От 3 до 6 лет        2   

                                          skills_raw     employer  
0  Вождение автомобилей представительского класса...  First-Class  
1  JavaScript, React, CSS, Node.js, TypeScript, W...      3VO inc  

Связей vacancy-skill: 11787
   vacancy_id                                      skill_name
0   120682290  Вождение автомобилей представительского класса

**Аналитика по зарплатам — сводные таблицы и графики**

В этом блоке строим аналитику по зарплатам — по городам, опыту, другим категориям. Данные пригодятся для сценариев Telegram-бота: "Сколько платят в городе X?", "Сколько получает junior/senior?" и др.

In [82]:
# Если salary_rub ещё не было рассчитано (унифицированная з/п), то рассчитываем:
if 'salary_rub' not in df_vacancy.columns:
    # Пример: если нет salary_rub, посчитать среднее из from/to, перевести в рубли
    def calc_salary(row):
        if pd.notnull(row['salary_from']) and pd.notnull(row['salary_to']):
            salary = (row['salary_from'] + row['salary_to']) / 2
        elif pd.notnull(row['salary_from']):
            salary = row['salary_from']
        elif pd.notnull(row['salary_to']):
            salary = row['salary_to']
        else:
            return None
        # Пример пересчёта валют (только если нужно, иначе убери этот блок)
        if row['salary_currency'] == 'RUR':
            return salary
        elif row['salary_currency'] == 'USD':
            return salary * 90  # Примерный курс
        elif row['salary_currency'] == 'EUR':
            return salary * 100
        else:
            return None
    df_vacancy['salary_rub'] = df_vacancy.apply(calc_salary, axis=1)

# Оставляем вакансии с зарплатой
df_salary = df_vacancy[df_vacancy['salary_rub'].notnull()].copy()
print(f"Вакансий с известной зарплатой: {len(df_salary)}")


Вакансий с известной зарплатой: 2423


In [83]:
# Группируем по area_id (регион) и считаем среднюю и медианную з/п
salary_by_area = df_salary.groupby('area_id')['salary_rub'].agg(['mean', 'median', 'count']).reset_index()
# Присоединяем наименования регионов
salary_by_area = salary_by_area.merge(df_area[['area_id', 'area_name']], on='area_id', how='left')
# Топ-10 городов по средней зарплате (минимум 10 вакансий)
salary_by_area_top = salary_by_area[salary_by_area['count'] >= 10].sort_values('mean', ascending=False).head(10)

print(salary_by_area_top[['area_name', 'mean', 'median', 'count']])

# Визуализация
fig = px.bar(
    salary_by_area_top,
    x='area_name', y='mean',
    error_y='median',
    text='count',
    title='Топ-10 городов по средней зарплате (руб)'
)
fig.show()

# Аналогично — по опыту (experience_hh)
salary_by_exp = df_salary.groupby('experience_hh')['salary_rub'].agg(['mean', 'median', 'count']).reset_index()
print("\nСредние зарплаты по опыту:")
print(salary_by_exp)

         area_name           mean     median  count
0           Москва  199836.632787  150000.00    305
1  Санкт-Петербург  129465.167112  109000.00    374
2     Екатеринбург  114566.232019   90000.00    431
3      Новосибирск   98175.020047   81427.75    424
7  Нижний Новгород   91372.950113   75000.00    441
9           Самара   82381.914153   70000.00    431



Средние зарплаты по опыту:
        experience_hh           mean    median  count
0         Более 6 лет  231039.797101  177500.0    138
1           Нет опыта   77688.272476   67500.0    723
2  От 1 года до 3 лет  101424.395488   85000.0   1086
3       От 3 до 6 лет  168946.101891  130000.0    476


In [84]:
import pandas as pd

# 1. Считываем вакансии и связи vacancy-skill из базы (или из файлов)
# (Если данные уже в переменных df_vacancy и df_vacancy_skill — пропусти этот шаг)
df_vacancy = pd.read_csv('/content/drive/MyDrive/hh-hr-bot/data/raw/feature_vacancies_new.csv')
df_vacancy_skill = pd.read_sql('SELECT * FROM vacancy_skill', con)  # если через duckdb

# 2. Объединяем таблицы для анализа: добавляем зарплату к каждому навыку
# Сначала приведём названия вакансий к строке, чтобы избежать ошибок
df_vacancy['id'] = df_vacancy['id'].astype(str)
df_vacancy_skill['vacancy_id'] = df_vacancy_skill['vacancy_id'].astype(str)

df_skills_salary = pd.merge(
    df_vacancy_skill,
    df_vacancy[['id', 'salary_rub']],
    left_on='vacancy_id', right_on='id',
    how='inner'
)
df_skills_salary = df_skills_salary.dropna(subset=['salary_rub'])  # Только с известной зарплатой

# 3. Группируем по навыку: считаем среднюю зарплату и частоту появления
top_skills_stats = (
    df_skills_salary
    .groupby('skill_name')
    .agg(
        count=('vacancy_id', 'count'),
        mean_salary=('salary_rub', 'mean'),
        median_salary=('salary_rub', 'median')
    )
    .sort_values('count', ascending=False)
    .reset_index()
)

# 4. Смотрим топ-10 навыков по частоте и по зарплате
print("Топ-10 самых популярных навыков (по количеству вакансий):")
print(top_skills_stats.head(10))

print("\nТоп-10 самых высокооплачиваемых навыков (указано минимум 10 вакансий):")
print(top_skills_stats[top_skills_stats['count'] >= 10].sort_values('mean_salary', ascending=False).head(10))

Топ-10 самых популярных навыков (по количеству вакансий):
                            skill_name  count    mean_salary  median_salary
0                     Работа в команде    149   84439.697987        75000.0
1                      Деловое общение    137  104069.379562        82250.0
2                    Деловая переписка    136  109682.029412        82500.0
3                      Ответственность    121   94428.421488        80000.0
4               Организаторские навыки    121  154011.644628       100000.0
5                      Пользователь ПК    113   75736.070796        62500.0
6                Телефонные переговоры    105  118380.952381        95000.0
7                       Грамотная речь    105   83127.314286        70000.0
8  Работа с большим объемом информации     89   93160.561798        70000.0
9                     Активные продажи     77  141548.701299       110000.0

Топ-10 самых высокооплачиваемых навыков (указано минимум 10 вакансий):
                       skill_name 

<ipython-input-84-984c1d958b62>:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Код для топ-5 навыков по количеству вакансий и по средней зарплате для города:

In [24]:
import pandas as pd

# df_vacancy — датафрейм вакансий (из таблицы vacancy)
# df_vacancy_skill — датафрейм связей vacancy_skill

# Склеиваем две таблицы
df_merge = df_vacancy_skill.merge(df_vacancy, left_on='vacancy_id', right_on='id')

# Теперь можно делать любые группировки!

# Пример: топ-10 навыков по Москве (area_id == 1)
top_skills_moscow = (
    df_merge[df_merge['area_id'] == 1]
    .groupby('skill_name')
    .agg(
        count=('vacancy_id', 'size'),
        mean_salary=('salary_rub', 'mean'),
        median_salary=('salary_rub', 'median')
    )
    .reset_index()
    .sort_values('count', ascending=False)
    .head(10)
)
print(top_skills_moscow)

# Пример: топ-10 навыков для middle (exp_middle == 1)
top_skills_middle = (
    df_merge[df_merge['exp_middle'] == 1]
    .groupby('skill_name')
    .agg(
        count=('vacancy_id', 'size'),
        mean_salary=('salary_rub', 'mean'),
        median_salary=('salary_rub', 'median')
    )
    .reset_index()
    .sort_values('count', ascending=False)
    .head(10)
)
print(top_skills_middle)

                                   skill_name  count    mean_salary  \
297                           Деловое общение     23  157505.000000   
295                         Деловая переписка     19  192730.769231   
474                    Организаторские навыки     16  418125.000000   
497                           Ответственность     15  194590.909091   
287                            Грамотная речь     15  113730.769231   
330              Знание устройства автомобиля     14  143076.923077   
294                      Деловая коммуникация     14  241000.000000   
343  Исполнение личных поручений руководителя     14  197772.727273   
592                          Работа в команде     13   91410.000000   
217                           Английский язык     13  353611.111111   

     median_salary  
297       107500.0  
295       120000.0  
474       187500.0  
497       120000.0  
287        95000.0  
330       150000.0  
294       120000.0  
343       165000.0  
592        77500.0  
217     

**Код для подсчёта топ-5 навыков по каждому уровню опыта**

In [26]:
import pandas as pd

# Загрузка данных из базы/файлов
vacancy = pd.read_csv('data/raw/feature_vacancies_new.csv')  # если нужен свежий датасет
vac_skill = con.execute("SELECT * FROM vacancy_skill").fetchdf()  # если прямо из DuckDB

# Мёржим skills с опытом
vac_skill_exp = vac_skill.merge(vacancy[['id', 'experience_hh']], left_on='vacancy_id', right_on='id')

# Группируем по опыту и навыку, считаем количество вакансий
skill_exp_counts = (
    vac_skill_exp
    .groupby(['experience_hh', 'skill_name'])
    .size()
    .reset_index(name='count')
)

# Для каждого опыта берём топ-5 навыков
top5_skills_by_exp = (
    skill_exp_counts
    .sort_values(['experience_hh', 'count'], ascending=[True, False])
    .groupby('experience_hh')
    .head(5)
)

# Если нужен красивый вывод:
for exp, group in top5_skills_by_exp.groupby('experience_hh'):
    print(f"\n==== {exp} ====")
    print(group[['skill_name', 'count']].to_string(index=False))



==== Более 6 лет ====
                                    skill_name  count
                  Знание устройства автомобиля     14
                             Деловая переписка     10
                      Бухгалтерская отчетность      9
        Водительское удостоверение категории B      9
Вождение автомобилей представительского класса      9

==== Нет опыта ====
      skill_name  count
Работа в команде     71
  Грамотная речь     55
 Ответственность     55
 Пользователь ПК     52
 Деловое общение     51

==== От 1 года до 3 лет ====
            skill_name  count
     Деловая переписка     82
       Деловое общение     76
      Работа в команде     68
Организаторские навыки     61
 Телефонные переговоры     58

==== От 3 до 6 лет ====
              skill_name  count
  Организаторские навыки     44
 Руководство коллективом     23
    Управление продажами     23
Бухгалтерская отчетность     22
    Деловая коммуникация     22


In [27]:
# Мёржим skills с опытом и зарплатой
vac_skill_exp_sal = vac_skill.merge(
    vacancy[['id', 'experience_hh', 'salary_rub']], left_on='vacancy_id', right_on='id'
)

# Группируем: опыт + навык, считаем число вакансий и среднюю/медианную зарплату
skill_exp_salary = (
    vac_skill_exp_sal
    .groupby(['experience_hh', 'skill_name'])
    .agg(count=('vacancy_id', 'size'),
         mean_salary=('salary_rub', 'mean'),
         median_salary=('salary_rub', 'median'))
    .reset_index()
)

# Для каждого грейда — топ-5 по популярности навыков с зарплатой
top5_skills_salary_by_exp = (
    skill_exp_salary
    .sort_values(['experience_hh', 'count'], ascending=[True, False])
    .groupby('experience_hh')
    .head(5)
)

# Выводим красиво
for exp, group in top5_skills_salary_by_exp.groupby('experience_hh'):
    print(f"\n==== {exp} ====")
    print(group[['skill_name', 'count', 'mean_salary', 'median_salary']].to_string(index=False, float_format="%.0f"))



==== Более 6 лет ====
                                    skill_name  count  mean_salary  median_salary
                  Знание устройства автомобиля     14       106358         112500
                             Деловая переписка     10       249286         200000
                      Бухгалтерская отчетность      9       137500         150000
        Водительское удостоверение категории B      9       112662         112500
Вождение автомобилей представительского класса      9       115900         135000

==== Нет опыта ====
      skill_name  count  mean_salary  median_salary
Работа в команде     71        63733          60000
  Грамотная речь     55        73369          65000
 Ответственность     55        70840          64000
 Пользователь ПК     52        58225          55000
 Деловое общение     51        79568          67500

==== От 1 года до 3 лет ====
            skill_name  count  mean_salary  median_salary
     Деловая переписка     82       109693          95000
      

In [32]:
df_vacancy = con.execute("SELECT * FROM vacancy").fetchdf()
# Группируем вакансии по работодателю (employer), считаем количество и среднюю зарплату
employer_stats = (
    df_vacancy.groupby('employer')
    .agg(count=('id', 'size'), mean_salary=('salary_rub', 'mean'), median_salary=('salary_rub', 'median'))
    .reset_index()
    .sort_values('count', ascending=False)
)

# Оставляем только топ-10
top10_employers = employer_stats.head(10)

print("Топ-10 работодателей по количеству вакансий:")
print(top10_employers.to_string(index=False, float_format="%.0f"))


KeyError: "Column(s) ['salary_rub'] do not exist"

In [92]:
import pandas as pd

def analyze_vacancy_vs_market(
    vacancy_id,
    df_vacancy,
    df_vacancy_skill,  # <-- именно таблица связи (vacancy_id, skill_name)
    df_area=None,
    top_n_skills=10
):
    #print(f"[LOG] Анализируем вакансию: {vacancy_id}")

    # 1. Найдём вакансию
    vac_row = df_vacancy[df_vacancy['id'] == vacancy_id]
    print(f"[LOG] Найдено строк в df_vacancy: {len(vac_row)}")
    if vac_row.empty:
        print("[ERROR] Вакансия не найдена в df_vacancy")
        return f"Вакансия с id {vacancy_id} не найдена."
    vac_row = vac_row.iloc[0]
    #print(f"[LOG] vac_row: {vac_row.to_dict()}")

    # 2. Определяем город/регион (по area_id) и опыт (грейд)
    if df_area is not None:
        city_list = df_area[df_area['area_id'] == vac_row['area_id']]['area_name'].values
        city = city_list[0] if len(city_list) > 0 else f"id={vac_row['area_id']}"
        print(f"[LOG] Определён город: {city}")
    else:
        city = vac_row['area_id']
    exp = vac_row['experience_hh']
    print(f"[LOG] Опыт (грейд): {exp}")
    salary = vac_row.get('salary_rub', None)
    #print(f"[LOG] Зарплата (salary_rub): {salary}")

    # 3. Формируем срез рынка по городу и опыту
    market_slice = df_vacancy[
        (df_vacancy['area_id'] == vac_row['area_id']) &
        (df_vacancy['experience_hh'] == exp) &
        (df_vacancy['salary_rub'].notnull())
    ]
    #print(f"[LOG] Размер market_slice: {len(market_slice)}")
    if market_slice.empty:
        print("[ERROR] Нет рыночных данных для такого города/грейда")
        return "Нет данных по рынку для этого города/грейда."

    # 4. Рыночные метрики
    median_market_salary = market_slice['salary_rub'].median()
    mean_market_salary = market_slice['salary_rub'].mean()
   # print(f"[LOG] Рыночные зарплаты: median={median_market_salary}, mean={mean_market_salary}")

    # 5. Сравнение зарплаты
    if pd.notnull(salary):
        if salary > median_market_salary:
            verdict = "Зарплата выше рынка"
        elif salary < median_market_salary:
            verdict = "Зарплата ниже рынка"
        else:
            verdict = "Зарплата примерно на уровне рынка"
    else:
        verdict = "Вакансия без указанной зарплаты"
    #print(f"[LOG] Вердикт: {verdict}")

    # 6. Навыки по вакансии
    # Получаем навыки из таблицы связей vacancy_skill
    # ВАЖНО! vacancy_id должен быть такого же типа как в df_vacancy_skill
    v_id_type = type(df_vacancy_skill['vacancy_id'].iloc[0])
    #print(f"[LOG] Типы: vacancy_id={type(vacancy_id)}, в таблице={v_id_type}")
    vac_id_casted = vacancy_id if isinstance(vacancy_id, v_id_type) else v_id_type(vacancy_id)
    vacancy_skills = df_vacancy_skill[df_vacancy_skill['vacancy_id'] == vac_id_casted]['skill_name'].str.lower().tolist()
    #print(f"[LOG] Навыков в вакансии: {len(vacancy_skills)} — {vacancy_skills}")

    # Топ-N по рынку (по количеству встречаемости в market_slice)
    top_skills = (
        df_vacancy_skill[df_vacancy_skill['vacancy_id'].isin(market_slice['id'])]
        .groupby('skill_name').size().sort_values(ascending=False).head(top_n_skills)
        .index.str.lower()
    )
   # print(f"[LOG] Топ-{top_n_skills} навыков по рынку: {list(top_skills)}")

    common_skills = set(vacancy_skills).intersection(set(top_skills))
    #print(f"[LOG] Совпадающих навыков: {len(common_skills)} — {common_skills}")

    # 7. Финальный отчёт
    report = (
        f"📊 Анализ вакансии #{vacancy_id}:\n"
        f"Город: {city}, опыт: {exp}\n"
        f"Ваша зарплата: {salary if salary is not None else 'не указана'} руб.\n"
        f"Медианная зарплата по рынку: {median_market_salary:.0f} руб.\n"
        f"Средняя зарплата по рынку: {mean_market_salary:.0f} руб.\n"
        f"▶️ {verdict}\n"
        f"\nНавыков в вакансии: {len(vacancy_skills)}\n"
        f"Совпадает с топ-{top_n_skills} рынка: {len(common_skills)}\n"
        f"Ваши совпадающие навыки: {', '.join(common_skills) if common_skills else 'нет'}\n"
    )
    print("[LOG] --- Отчёт сформирован ---")
    return report

# ---- Пример использования ----
analyze_vacancy_vs_market(str(120682290), df_vacancy, df_vacancy_skill, df_area)







[LOG] Найдено строк в df_vacancy: 1
[LOG] Определён город: Екатеринбург
[LOG] Опыт (грейд): Более 6 лет
[LOG] --- Отчёт сформирован ---


'📊 Анализ вакансии #120682290:\nГород: Екатеринбург, опыт: Более 6 лет\nВаша зарплата: 90000.0 руб.\nМедианная зарплата по рынку: 190000 руб.\nСредняя зарплата по рынку: 230679 руб.\n▶️ Зарплата ниже рынка\n\nНавыков в вакансии: 5\nСовпадает с топ-10 рынка: 3\nВаши совпадающие навыки: знание устройства автомобиля, водительское удостоверение категории b, вождение автомобилей представительского класса\n'

In [86]:
print(df_vacancy[df_vacancy['id'] == vacancy_id])
print(df_salary[df_salary['id'] == vacancy_id])
print(df_vacancy_skill[df_vacancy_skill['vacancy_id'] == vacancy_id])


          id                          title              published_at  \
0  120682290  Водитель с личным автомобилем  2025-05-19T09:30:49+0300   

                                         description  salary_from  salary_to  \
0  <p>Вакансия &quot;Водитель с личным автомобиле...          NaN    90000.0   

  salary_currency experience_hh  area_id  \
0             RUR   Более 6 лет        3   

                                          skills_raw     employer  salary_rub  \
0  Вождение автомобилей представительского класса...  First-Class     90000.0   

   desc_len  desc_words  title_len  num_skills  exp_junior  exp_middle  \
0      1397         155         29           5           0           0   

   exp_senior  exp_lead  
0           0         1  
          id                          title              published_at  \
0  120682290  Водитель с личным автомобилем  2025-05-19T09:30:49+0300   

                                         description  salary_from  salary_to  \
0  <p>Ваканс

In [64]:
vacancy_id = str(120682290)  # Обрати внимание: возможно, это строка!
report = analyze_vacancy_vs_market(vacancy_id, df_salary, df_area)
print(report)


TypeError: analyze_vacancy_vs_market() missing 1 required positional argument: 'df_salary'